In [1]:
import torch
from torch import nn
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms
import pandas as pd

In [2]:
class Network(nn.Module):
    def __init__(self, p1, p2):
        super().__init__()
        self.features = nn.Sequential(
            nn.Linear(31, 25, bias= True),
            nn.ReLU(),
            nn.Linear(25, 20, bias= True),
            nn.ReLU(),
            nn.Dropout(p= p1),
            nn.Linear(20, 15, bias = True),
            nn.ReLU(),
            nn.Dropout(p= p2),
            nn.Linear(15, 10, bias= True),
            nn.ReLU()
        )
        self.classify = nn.Sequential(
            nn.Linear(10, 1, bias= True),
            nn.Sigmoid()
        )
    
    def forward(self, x):
        return self.classify(self.features(x))

In [3]:
class Data(Dataset):
    def __init__(self, x, y):
        super().__init__()
        self.x = torch.tensor(x.values, dtype= torch.float32)
        self.y = torch.tensor(y.values, dtype= torch.float32)
    def __len__(self):
        return self.x.shape[0]
    def __getitem__(self, index):
        x, y = self.x[index], self.y[index]
        return x, y

In [4]:
xtrain, ytrain, xtest, ytest = pd.read_csv('./X_train.csv', header= None, index_col = None), pd.read_csv('./y_train.csv', header = None, index_col = None), pd.read_csv('./X_test.csv', header = None, index_col = None), pd.read_csv('./y_test.csv', header = None, index_col = None)
train_data = Data(x= xtrain, y= ytrain)
test_data = Data(x= xtest, y= ytest)
train_loader = DataLoader(train_data, batch_size=10000, shuffle= True)
test_loader = DataLoader(test_data, batch_size= 10000)

In [5]:
model = Network(0.5, 0.5)
model.to('cuda')

Network(
  (features): Sequential(
    (0): Linear(in_features=31, out_features=25, bias=True)
    (1): ReLU()
    (2): Linear(in_features=25, out_features=20, bias=True)
    (3): ReLU()
    (4): Dropout(p=0.5, inplace=False)
    (5): Linear(in_features=20, out_features=15, bias=True)
    (6): ReLU()
    (7): Dropout(p=0.5, inplace=False)
    (8): Linear(in_features=15, out_features=10, bias=True)
    (9): ReLU()
  )
  (classify): Sequential(
    (0): Linear(in_features=10, out_features=1, bias=True)
    (1): Sigmoid()
  )
)

In [6]:
criterion = nn.BCELoss()
optimizer = torch.optim.Adam(model.parameters(), lr = 0.01)

In [7]:
for epoch in range(100):
    model.train()
    running_loss = 0.0
    for input, target in train_loader:
        input, target = input.to('cuda'), target.to('cuda')
        optimizer.zero_grad()
        output = model(input)
        loss = criterion(output, target)
        loss.backward()
        optimizer.step()
        running_loss += loss.item()
    print(f'loss - {running_loss}')

loss - 63.609829127788544
loss - 33.837638795375824
loss - 33.824039697647095
loss - 33.83064126968384
loss - 33.824156165122986
loss - 33.826070964336395
loss - 33.828673183918
loss - 33.827759861946106
loss - 33.817284405231476
loss - 33.808863401412964
loss - 33.81495827436447
loss - 33.8258353471756
loss - 33.824654936790466
loss - 33.81456524133682
loss - 33.82159596681595
loss - 33.81598216295242
loss - 33.81394696235657
loss - 33.814747989177704
loss - 33.811072289943695
loss - 33.80386120080948
loss - 33.80431580543518
loss - 33.80136811733246
loss - 33.80238664150238
loss - 33.8123385310173
loss - 33.802214205265045
loss - 33.797362327575684
loss - 33.79121798276901
loss - 33.78795915842056
loss - 33.79780775308609
loss - 33.79899877309799
loss - 33.78781658411026
loss - 33.78034174442291
loss - 33.783567011356354
loss - 33.77648276090622
loss - 33.78610956668854
loss - 33.789334654808044
loss - 33.78431278467178
loss - 33.77244961261749
loss - 33.79081493616104
loss - 33.7797

In [9]:
model.eval()
all_preds, all_labels = [], []
with torch.no_grad():
    for input, target in test_loader:
        input, target = input.to('cuda'), target.to('cuda')
        output = torch.round(model(input))
        all_preds.extend(output.to('cpu'))
        all_labels.extend(target.to('cpu'))

from sklearn.metrics import accuracy_score
print(accuracy_score(all_preds, all_labels))


0.6264376052452438
